In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from glob import glob
from sklearn.externals import joblib
from scipy.stats import norm
%matplotlib inline

In [2]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

---

In [2]:
import keras

Using TensorFlow backend.


In [4]:
model_tmp = keras.models.load_model('models/model_audio_phase2.h5')
model_audio = keras.models.Model(model_tmp.input, model_tmp.get_layer('dropout_2').output)

In [5]:
model_tmp = keras.models.load_model('models/model_eyes_phase2.h5')
model_eyes = keras.models.Model(model_tmp.input, model_tmp.get_layer('dropout_2').output)

In [6]:
model_tmp = keras.models.load_model('models/model_face_phase2.h5')
model_face = keras.models.Model(model_tmp.input, model_tmp.get_layer('dropout_2').output)

In [7]:
model_tmp = keras.models.load_model('models/model_kinect_phase2.h5')
model_kinect = keras.models.Model(model_tmp.input, model_tmp.get_layer('dropout_2').output)

In [8]:
del model_tmp

In [9]:
def norm(arr, mins, maxs):
    arr = arr.astype(float)
    for i in range(arr.shape[-1]):
        arr[:, :, i] -= mins[i]
        arr[:, :, i] /= maxs[i]
    return arr

In [10]:
mins_audio, maxs_audio = np.load('data/audio_mins.npy'), np.load('data/audio_maxs.npy')
mins_kinect, maxs_kinect = np.load('data/kinect_mins.npy'), np.load('data/kinect_maxs.npy')

In [15]:
f_videos = sorted(glob('data/X_test/*.npy'))

print len(f_videos)

141


In [27]:
for f in log_progress(f_videos):
    X = np.load(f).item()
    
    print X['audio'][np.isnan(X['audio'])].shape, X['eyes'][np.isnan(X['eyes'])].shape, \
          X['face_nn'][np.isnan(X['face_nn'])].shape, X['kinect'][np.isnan(X['kinect'])].shape

VBox(children=(HTML(value=u''), IntProgress(value=0, max=141)))

(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)
(0,) (0,) (0,) (0,)


In [19]:
for f in log_progress(videos):
    X = np.load(f).item()
#     print X['audio'].shape, X['eyes'].shape, X['face_nn'].shape, X['kinect'].shape
    
    X_audio = norm(X['audio'], mins_audio, maxs_audio)
    X_eyes = X['eyes']
    X_face = X['face_nn']
    X_kinect = norm(X['kinect'], mins_kinect, maxs_kinect)
    
    nn_audio = model_audio.predict(X_audio)
    nn_eyes = model_eyes.predict(X_eyes)
    nn_face = model_face.predict(X_face)
    nn_kinect = model_kinect.predict(X_kinect)
#     print nn_audio.shape, nn_eyes.shape, nn_face.shape, nn_kinect.shape
    
    embd = np.concatenate([nn_audio, nn_eyes, nn_face, nn_kinect], axis=1)
    print embd.shape
    
    np.save('out/{}'.format(os.path.basename(f)), embd)
    
#     break

VBox(children=(HTML(value=u''), IntProgress(value=0, max=141)))

(5533, 102)
(2704, 102)
(2752, 102)
(2879, 102)
(2477, 102)
(2821, 102)
(2834, 102)
(2886, 102)
(8107, 102)
(2821, 102)
(3773, 102)
(7147, 102)
(2704, 102)
(4316, 102)
(2457, 102)
(4575, 102)
(3382, 102)
(706, 102)
(5472, 102)
(2692, 102)
(862, 102)
(5328, 102)
(5090, 102)
(3467, 102)
(4280, 102)
(2303, 102)
(5482, 102)
(5061, 102)
(2879, 102)
(2303, 102)
(5646, 102)
(3571, 102)
(2752, 102)
(3285, 102)
(4205, 102)
(3550, 102)
(3271, 102)
(4631, 102)
(4436, 102)
(5123, 102)
(7794, 102)
(4331, 102)
(878, 102)
(1557, 102)
(2538, 102)
(5646, 102)
(5242, 102)
(2834, 102)
(3938, 102)
(5328, 102)
(1341, 102)
(2567, 102)
(3361, 102)
(5123, 102)
(706, 102)
(3382, 102)
(3285, 102)
(5090, 102)
(4687, 102)
(3853, 102)
(2538, 102)
(4552, 102)
(6309, 102)
(3906, 102)
(4170, 102)
(3592, 102)
(4687, 102)
(3361, 102)
(4205, 102)
(3024, 102)
(3938, 102)
(1989, 102)
(4680, 102)
(5472, 102)
(4680, 102)
(1282, 102)
(2692, 102)
(5397, 102)
(3142, 102)
(3271, 102)
(1694, 102)
(4699, 102)
(2567, 102)
(2477, 1

---

In [3]:
LABELS_TEST_PATH = '/home/datasets/merc-2017/test/prediction/'
OUT_PATH = 'submission4'

In [4]:
predictor = joblib.load('models/predictor_phase2.pkl')

In [5]:
videos = sorted(os.listdir('out'))

print len(videos)

141


In [6]:
X_test_tinds = np.load('data/X_test_tinds.npy').item()

print len(X_test_tinds)

141


In [7]:
if os.path.exists(OUT_PATH):
    print 'Submission already exists! Aborting.'
else:
    os.makedirs(os.path.join(OUT_PATH, 'prediction'))
    
    for video in log_progress(videos[:]):
        video = video.split('.npy')[0]

        embd = np.load('out/' + video + '.npy')
        scores = predictor.predict_proba(embd)
        tinds = np.asarray(X_test_tinds[video + '.csv'])
        df = pd.read_csv(os.path.join(LABELS_TEST_PATH, video + '.csv'))

        for t_i, t in df.Time.iteritems():
            tmp_timings = tinds[(tinds[:, 0] <= t) & (tinds[:, 1] >= t)]
            tmp_scores = scores[(tinds[:, 0] <= t) & (tinds[:, 1] >= t)]
            
            # s3
            ################################################
#             hlabels = np.argmax(tmp_scores, axis=1).tolist()
#             y_pred = np.argmax([hlabels.count(i) for i in range(6)])
#             df.iloc[t_i, y_pred + 1] = 1.0
#             continue
            ################################################

            if len(tmp_timings) == 0:
                if t_i > 0:
                    df.iloc[t_i, y_pred + 1] = 1.0
                else:
                    print 'Warning!'
                continue

            tavgs = np.mean(tmp_timings, axis=1)
            t_closest_ind = np.argmin(np.abs(tavgs - t))

            if len(tmp_timings) == 1:
                weights = np.asarray([1.0])
            else:
                # s1
                ################################################
#                 weights = norm.pdf(tavgs, loc=tavgs[t_closest_ind], scale=np.abs(tavgs - tavgs[t_closest_ind]).max()/3.0)
                ################################################
                # s2
                ################################################
#                 weights = np.asarray([1.0] * len(tavgs))
                ################################################
                # s4
                ################################################
                weights = norm.pdf(tavgs, loc=tavgs[-1], scale=(tavgs[-1] - tavgs[0])/3.0)
                ################################################
        
                weights /= weights.max()

            tmp_scores_w = np.sum((tmp_scores.T * weights.T).T, axis=0)
            y_pred = np.argmax(tmp_scores_w)
#             print y_pred

            df.iloc[t_i, y_pred + 1] = 1.0

        df.to_csv(os.path.join(OUT_PATH, 'prediction', video + '.csv'), index=False)

#         break

VBox(children=(HTML(value=u''), IntProgress(value=0, max=141)))